In [75]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190114_002018_meshID_444481be-5cba-4dad-a316-b52203791b61'
l = elastic_rods.RodLinkage('../examples/20190114_002018_meshID_444481be-5cba-4dad-a316-b52203791b61.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [15, 3])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.setCameraParams(((1.817742369957575, 1.9774244735965147, 1.0379653320383124),
 (-0.18560075634960396, -0.1740407238201695, 0.9670895437830958),
 (-0.12349092883459574, 0.12349092883459574, -1.0388992503077217)))
view.show()


Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [58]:
view.getCameraParams()

((1.817742369957575, 1.9774244735965147, 1.0379653320383124),
 (-0.18560075634960396, -0.1740407238201695, 0.9670895437830958),
 (-0.12349092883459574, 0.12349092883459574, -1.0388992503077217))

In [ ]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [ ]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [76]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-12
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [77]:
import random as rnd
rl = l.getPerSegmentRestLength()

for index in range(0,len(rl)):
    print (rl[index])
    randomVar=(rnd.uniform(-2,2))
    rl[index]=rl[index]+randomVar
    print (rl[index])


#customLenghtIndices=[172,175,174,173]

#for index in customLenghtIndices:
#    rl[index]=rl[index]*1.2

l.setPerSegmentRestLength(rl)

121.5236447347353
122.98920528589085
95.10880586419783
94.8516680562867
172.46611424882377
171.40730434621432
145.8630243394159
143.88484400573736
95.49357120560339
94.34496724283761
69.51395478933956
67.52910669986731
121.48539000484057
121.60713207525961
150.03912520998185
149.74257043125093
262.28967593071343
263.3915969127238
239.8698674694307
241.10226648154696
351.5251525020849
352.8540220114299
334.93018382794145
335.8284402877832
256.7829700023659
256.78022303321234
98.01278392052336
96.66197791833855
173.75081279404594
175.38308499362768
248.31523768448463
248.86680882562655
364.8776925615188
365.878271952305
351.1775278006148
352.3019043721017
351.2668182382055
353.18308408542106
351.3832652669099
350.1428678935836
155.44501943122197
154.2041786529076
266.6296066662907
268.2464726403474
364.60053096142883
365.5169366793694
364.67589183654763
364.4288151847872
364.7628820861047
365.6965994151456
248.38058932020277
248.6684396840095
262.1512308679283
261.9860322507961
239.92648

In [78]:
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(62) - l.averageJointAngle, 10, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  0.20790957433750967
CHOLMOD detected non-positive definite matrix!; increasing tau to 1e-08
CHOLMOD detected non-positive definite matrix!; increasing tau to 4e-08
CHOLMOD detected non-positive definite matrix!; increasing tau to 1.6e-07
CHOLMOD detected non-positive definite matrix!; increasing tau to 6.4e-07
CHOLMOD detected non-positive definite matrix!; increasing tau to 2.56e-06
CHOLMOD detected non-positive definite matrix!; increasing tau to 1.024e-05
CHOLMOD detected non-positive definite matrix!; increasing tau to 4.096e-05
CHOLMOD detected non-positive definite matrix!; increasing tau to 0.00016384
CHOLMOD detected non-positive definite matrix!; increasing tau to 0.00065536
CHOLMOD detected non-positive definite matrix!; increasing tau to 0.00262144
CHOLMOD detected non-positive definite matrix!; increasing tau to 0.0104858
CHOLMOD detected non-positive definite matrix!; increasing tau to 0.041943
CHOLMOD detected non-positive definite matrix!; increasing tau t

In [ ]:
from write_render_files import writeRenderFiles
writeRenderFiles(l, 'Boat', 'the_boat_depl')